In [1]:
!pip install tensorflow

In [2]:
import pandas as pd
import tensorflow as tf
import sklearn

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
#Com base nos atributos do carro, vamos prever o preço
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')

In [4]:
base

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000.0,0.0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0.0,70435.0,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000.0,5.0,diesel,audi,ja,2016-03-24 00:00:00,0.0,66954.0,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000.0,8.0,diesel,jeep,NaN,2016-03-14 00:00:00,0.0,90480.0,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000.0,6.0,benzin,volkswagen,nein,2016-03-17 00:00:00,0.0,91074.0,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000.0,7.0,diesel,skoda,nein,2016-03-31 00:00:00,0.0,60437.0,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210474,2016-03-21 13:42:12,Mercedes_230_CE_Liebhaber_Coupe,privat,Angebot,1350,test,coupe,1991,manuell,132,e_klasse,150000.0,8.0,benzin,mercedes_benz,nein,2016-03-21 00:00:00,0.0,78467.0,2016-04-06 08:45:56
210475,2016-03-25 11:57:01,Volkswagen_Scirocco,privat,Angebot,6999,test,coupe,1978,manuell,69,scirocco,125000.0,1.0,benzin,volkswagen,ja,2016-03-25 00:00:00,0.0,86368.0,2016-03-30 01:15:24
210476,2016-03-09 02:02:46,Volkswagen_Golf_2.8_V6_4Motion,privat,Angebot,7500,control,limousine,1999,manuell,204,golf,150000.0,9.0,benzin,volkswagen,nein,2016-03-09 00:00:00,0.0,31552.0,2016-03-22 01:18:29
210477,2016-03-24 05:36:23,Polo_6n2_1.4_Sitzheizung_Klima_TÜV_12/17,privat,Angebot,1499,control,kleinwagen,2000,manuell,60,polo,150000.0,5.0,benzin,volkswagen,nein,2016-03-24 00:00:00,0.0,58644.0,2016-03-25 17:15:29


In [5]:
#apagar colunas irrelevantes - analisar cada atributo separadamente e decidir se é possível utilizá-lo ou não
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)
base

,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000.0,0.0,benzin,volkswagen,NaN
1,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000.0,5.0,diesel,audi,ja
2,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000.0,8.0,diesel,jeep,NaN
3,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000.0,6.0,benzin,volkswagen,nein
4,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000.0,7.0,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210474,Mercedes_230_CE_Liebhaber_Coupe,privat,Angebot,1350,test,coupe,1991,manuell,132,e_klasse,150000.0,8.0,benzin,mercedes_benz,nein
210475,Volkswagen_Scirocco,privat,Angebot,6999,test,coupe,1978,manuell,69,scirocco,125000.0,1.0,benzin,volkswagen,ja
210476,Volkswagen_Golf_2.8_V6_4Motion,privat,Angebot,7500,control,limousine,1999,manuell,204,golf,150000.0,9.0,benzin,volkswagen,nein
210477,Polo_6n2_1.4_Sitzheizung_Klima_TÜV_12/17,privat,Angebot,1499,control,kleinwagen,2000,manuell,60,polo,150000.0,5.0,benzin,volkswagen,nein


In [6]:
base['name'].value_counts()

,count
name,
Ford_Fiesta,378
Opel_Corsa,350
Volkswagen_Golf_1.4,346
BMW_318i,337
BMW_316i,298
...,...
Polo_6N_zuverlaessiger_4_Tuerer_mit_einem_Jahr_TÜV,1
Honda_Jazz_1.4__**Klima**AHK**,1
Volkswagen_VW_Passat_1.8_5V_Turbo_erst_134000km_Tuev_Neu_Top,1


In [ ]:
#Vamos apagar a coluna de nome, pois há muitos valores unicos, impedindo o treinamento do modelo

In [7]:
base = base.drop('name', axis=1)

In [8]:
base['seller'].value_counts()

,count
seller,
privat,210476
gewerblich,3


In [9]:
base = base.drop('seller', axis=1)

In [10]:
base['offerType'].value_counts()

,count
offerType,
Angebot,210473
Gesuch,6


In [11]:
base = base.drop('offerType', axis=1)

In [12]:
base

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000.0,0.0,benzin,volkswagen,NaN
1,18300,test,coupe,2011,manuell,190,NaN,125000.0,5.0,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000.0,8.0,diesel,jeep,NaN
3,1500,test,kleinwagen,2001,manuell,75,golf,150000.0,6.0,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000.0,7.0,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
210474,1350,test,coupe,1991,manuell,132,e_klasse,150000.0,8.0,benzin,mercedes_benz,nein
210475,6999,test,coupe,1978,manuell,69,scirocco,125000.0,1.0,benzin,volkswagen,ja
210476,7500,control,limousine,1999,manuell,204,golf,150000.0,9.0,benzin,volkswagen,nein
210477,1499,control,kleinwagen,2000,manuell,60,polo,150000.0,5.0,benzin,volkswagen,nein


In [13]:
#analisando valores inconsistentes
base.loc[base.price <= 10]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
7,0,test,limousine,1980,manuell,50,andere,40000.0,7.0,benzin,volkswagen,nein
40,0,test,NaN,1990,NaN,0,corsa,150000.0,1.0,benzin,opel,NaN
60,1,control,suv,1994,manuell,286,NaN,150000.0,11.0,NaN,sonstige_autos,NaN
91,1,control,limousine,1995,manuell,113,e_klasse,150000.0,4.0,diesel,mercedes_benz,nein
115,0,test,NaN,2017,manuell,0,golf,5000.0,12.0,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
210327,0,test,andere,2006,manuell,0,NaN,20000.0,12.0,andere,sonstige_autos,ja
210332,1,control,kombi,2005,NaN,0,NaN,5000.0,0.0,benzin,mercedes_benz,NaN
210372,0,test,NaN,2005,NaN,0,a3,150000.0,0.0,NaN,audi,NaN
210418,0,control,suv,2003,automatik,150,andere,150000.0,12.0,diesel,hyundai,nein


In [14]:
base['price'].mean()

21816.002128478376

In [15]:
#apagar os registros em que o preço está invalido
base = base[base['price']>10]
base.shape

(203550, 12)

In [16]:
#filtrando preços muito altos
base.loc[base['price'] > 350000]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1846,579000,control,coupe,1980,manuell,277,andere,20000.0,12.0,benzin,bmw,nein
10649,420000,control,coupe,2004,manuell,483,911,50000.0,4.0,benzin,porsche,nein
14663,11111111,control,coupe,2003,manuell,64,polo,150000.0,2.0,benzin,volkswagen,NaN
16889,1000000,control,kombi,1998,NaN,0,mondeo,150000.0,0.0,benzin,ford,ja
20143,1250000,test,coupe,2016,manuell,500,911,5000.0,3.0,benzin,porsche,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
189737,99999999,control,kleinwagen,1956,manuell,15,NaN,150000.0,9.0,benzin,sonstige_autos,nein
194217,849000,test,kombi,1998,manuell,75,golf,150000.0,1.0,benzin,volkswagen,nein
195299,370000,control,coupe,2010,manuell,620,911,20000.0,12.0,benzin,porsche,nein
196306,10000000,control,NaN,2017,manuell,0,golf,5000.0,0.0,benzin,volkswagen,NaN


In [17]:
base = base.loc[base['price']<350000]
base.shape

(203475, 12)

In [18]:
base.loc[pd.isnull(base['vehicleType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000.0,0.0,benzin,volkswagen,NaN
16,300,test,NaN,2016,NaN,60,polo,150000.0,0.0,benzin,volkswagen,NaN
22,2900,test,NaN,2018,manuell,90,meriva,150000.0,5.0,benzin,opel,nein
26,5555,control,NaN,2017,manuell,125,c4,125000.0,4.0,NaN,citroen,nein
31,899,control,NaN,2016,manuell,60,clio,150000.0,6.0,benzin,renault,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
210433,5950,control,NaN,2017,manuell,120,astra,150000.0,1.0,diesel,opel,nein
210441,260,control,NaN,2000,NaN,0,fiesta,150000.0,0.0,NaN,ford,NaN
210444,1000,control,NaN,2017,manuell,85,lupo,150000.0,0.0,diesel,volkswagen,NaN
210449,2900,test,NaN,2017,NaN,0,twingo,125000.0,6.0,benzin,renault,NaN


In [19]:
#Para o tratamento de atributos faltantes em atributos numéricos: substituímos os faltantes pela média
#Para o tratamento de atributos faltantes em atributos categóricos: substituímos os faltantes pela moda
base['vehicleType'].value_counts()
#Nesse caso, vamos substituir os valores em branco por "limousine", que é o valor que mais se repete

,count
vehicleType,
limousine,52861
kleinwagen,44261
kombi,37328
bus,16782
cabrio,12867
coupe,10374
suv,8215
andere,1784


In [20]:
base.loc[pd.isnull(base['gearbox'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
15,450,test,kleinwagen,1910,NaN,0,ka,5000.0,0.0,benzin,ford,NaN
16,300,test,NaN,2016,NaN,60,polo,150000.0,0.0,benzin,volkswagen,NaN
32,245,test,limousine,1994,NaN,0,golf,150000.0,2.0,benzin,volkswagen,nein
37,1500,test,NaN,2016,NaN,0,kangoo,150000.0,1.0,diesel,renault,nein
70,1200,test,coupe,2001,NaN,0,astra,150000.0,0.0,NaN,opel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
210366,1100,control,NaN,2016,NaN,75,lupo,150000.0,0.0,benzin,volkswagen,NaN
210407,1000,test,NaN,2000,NaN,0,golf,150000.0,0.0,NaN,volkswagen,NaN
210409,790,control,NaN,2000,NaN,0,passat,30000.0,12.0,NaN,volkswagen,NaN
210441,260,control,NaN,2000,NaN,0,fiesta,150000.0,0.0,NaN,ford,NaN


In [21]:
base['gearbox'].value_counts()

,count
gearbox,
manuell,151059
automatik,42698


In [22]:
base.loc[pd.isnull(base['model'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,test,coupe,2011,manuell,190,NaN,125000.0,5.0,diesel,audi,ja
83,350,control,kleinwagen,1997,manuell,54,NaN,150000.0,3.0,NaN,fiat,ja
139,1450,control,limousine,1992,manuell,136,NaN,150000.0,0.0,NaN,audi,nein
156,6799,control,kleinwagen,2009,NaN,60,NaN,20000.0,5.0,benzin,volkswagen,nein
165,500,control,kleinwagen,1999,manuell,0,NaN,150000.0,0.0,benzin,renault,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
210375,1500,control,bus,1991,manuell,0,NaN,150000.0,0.0,NaN,volkswagen,NaN
210391,3500,test,limousine,2007,manuell,60,NaN,50000.0,11.0,benzin,seat,nein
210411,300,test,kombi,2000,manuell,75,NaN,150000.0,4.0,diesel,ford,NaN
210415,5300,test,limousine,2001,automatik,231,NaN,150000.0,7.0,benzin,bmw,nein


In [23]:
base['model'].value_counts()

,count
model,
golf,16242
andere,14575
3er,11266
polo,7200
corsa,6860
...,...
i3,6
rangerover,5
kappa,5


In [24]:
base.loc[pd.isnull(base['fuelType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
9,999,test,kleinwagen,1998,manuell,101,golf,150000.0,0.0,NaN,volkswagen,NaN
13,2500,control,kombi,2004,manuell,131,passat,150000.0,2.0,NaN,volkswagen,nein
26,5555,control,NaN,2017,manuell,125,c4,125000.0,4.0,NaN,citroen,nein
36,1600,control,andere,1991,manuell,75,kadett,70000.0,0.0,NaN,opel,NaN
41,7500,control,limousine,2002,automatik,306,e_klasse,150000.0,4.0,NaN,mercedes_benz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
210375,1500,control,bus,1991,manuell,0,NaN,150000.0,0.0,NaN,volkswagen,NaN
210407,1000,test,NaN,2000,NaN,0,golf,150000.0,0.0,NaN,volkswagen,NaN
210409,790,control,NaN,2000,NaN,0,passat,30000.0,12.0,NaN,volkswagen,NaN
210441,260,control,NaN,2000,NaN,0,fiesta,150000.0,0.0,NaN,ford,NaN


In [25]:
base['fuelType'].value_counts()

,count
fuelType,
benzin,123232
diesel,59985
lpg,2891
cng,328
hybrid,150
andere,100
elektro,63


In [26]:
base.loc[pd.isnull(base['notRepairedDamage'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000.0,0.0,benzin,volkswagen,NaN
2,9800,test,suv,2004,automatik,163,grand,125000.0,8.0,diesel,jeep,NaN
8,14500,control,bus,2014,manuell,125,c_max,30000.0,8.0,benzin,ford,NaN
9,999,test,kleinwagen,1998,manuell,101,golf,150000.0,0.0,NaN,volkswagen,NaN
12,999,control,kombi,1995,manuell,115,passat,150000.0,11.0,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
210449,2900,test,NaN,2017,NaN,0,twingo,125000.0,6.0,benzin,renault,NaN
210457,5999,test,bus,2008,manuell,143,NaN,125000.0,8.0,diesel,mazda,NaN
210468,700,control,bus,1991,manuell,0,transporter,150000.0,9.0,diesel,volkswagen,NaN
210471,500,test,kleinwagen,1997,manuell,50,polo,150000.0,9.0,benzin,volkswagen,NaN


In [27]:
base['notRepairedDamage'].value_counts()

,count
notRepairedDamage,
nein,146939
ja,19161


In [28]:
#definindo os valores do que vai ser substituído
valores = {'vehicleType':'limousine', 'gearbox':'manuell', 'model':'golf', 'fuelType':'benzin', 'notRepairedDamage':'nein'}

In [29]:
#Quando for encontrado um valor nulo, ele será substituído por um desses valores
base = base.fillna(value=valores)

In [30]:
base.isnull().sum()

,0
price,0
abtest,0
vehicleType,0
yearOfRegistration,0
gearbox,0
powerPS,0
model,0
kilometer,1
monthOfRegistration,1
fuelType,0


In [31]:
base.columns

Index(['price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage'],
      dtype='object')

In [32]:
#Agora que os dados estão tratados, vamos salvar os atributos previsores. Vamos especificar que X começa a partir da coluna 1, pois a coluna 0 corresponde ao preço, que é o que queremos prever
# o .value no final é para transformar em numpy
X = base.iloc[:,1:12].values
X

array([['test', 'limousine', 1993, ..., 'benzin', 'volkswagen', 'nein'],
       ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
       ['test', 'suv', 2004, ..., 'diesel', 'jeep', 'nein'],
       ...,
       ['control', 'limousine', 1999, ..., 'benzin', 'volkswagen',
        'nein'],
       ['control', 'kleinwagen', 2000, ..., 'benzin', 'volkswagen',
        'nein'],
       ['control', 'kombi', 2003, ..., 'benzin', nan, 'nein']],
      dtype=object)

In [33]:
y = base.iloc[:,0].values
y

array([  480, 18300,  9800, ...,  7500,  1499,  4990])

In [34]:
#As redes neurais não entendem variáveis categóricas, somente numéricas. Por isso, vamos fazer o one hot encoder, para cada categoria ter uma variavel especifica
base['brand'].value_counts()

,count
brand,
volkswagen,43271
bmw,22198
opel,21763
mercedes_benz,19552
audi,18043
ford,14023
renault,9887
peugeot,6106
fiat,5298


In [35]:
base.columns

Index(['price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage'],
      dtype='object')

In [40]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])], remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [41]:
X

array([[0.00e+00, 1.00e+00, 0.00e+00, ..., 0.00e+00, 1.50e+05, 0.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.90e+02, 1.25e+05, 5.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.63e+02, 1.25e+05, 8.00e+00],
       ...,
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 2.04e+02, 1.50e+05, 9.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 6.00e+01, 1.50e+05, 5.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 1.31e+02,      nan,      nan]])

In [42]:
base.shape

(203475, 12)

In [43]:
X.shape

(203475, 315)

In [44]:
#Vamos usar a função linear, pois é uma regressão
regressor = Sequential([
    tf.keras.layers.InputLayer(shape=(315,)),
    tf.keras.layers.Dense(units=158, activation='relu'),
    tf.keras.layers.Dense(units=158, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='linear')
])

In [45]:
regressor.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 158)                 │          49,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 158)                 │          25,122 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             159 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 75,209 (293.79 KB)

 Trainable params: 75,209 (293.79 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
regressor.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])

In [47]:
regressor.fit(X, y, batch_size=300, epochs=100)

Epoch 1/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: nan - mean_absolute_error: nan
Epoch 2/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: nan - mean_absolute_error: nan
Epoch 3/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: nan - mean_absolute_error: nan
Epoch 4/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: nan - mean_absolute_error: nan
Epoch 5/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: nan - mean_absolute_error: nan
Epoch 6/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: nan - mean_absolute_error: nan
Epoch 7/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: nan - mean_absolute_error: nan
Epoch 8/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: nan - mean_absolute_error: nan
Epoch 9/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: nan - mean_absolute_error: nan
Epoch 10/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: nan - mean_absolute_error: nan
Epoch 11/100
679/679 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: nan - mean_absolu

In [48]:
X.shape

(203475, 315)

In [49]:
previsoes = regressor.predict(X)

6359/6359 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step


In [50]:
y.mean()

5939.067389114141

In [52]:
previsoes.mean()

nan